In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(numbers, max_num):
        def extraction():
            rnd.shuffle(whole_numbers := list(range(1, max_num+1)))
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return [extraction() for _ in range(numbers)]

    @staticmethod
    def sample(numbers, max_num):
        whole_numbers = list(range(1, max_num+1))

        return rnd.sample(whole_numbers, k=numbers)

    def int(self, numbers, max_num):
        combo = set()
        while len(combo) < numbers:
            combo.add(next(self.pallottoliere(max_num)))

        return combo

    @staticmethod
    def many_(backend, numbers, max_num):
        samples = [backend(numbers, max_num) for _ in range(max_num)]
        rnd.shuffle(samples)

        return rnd.choice(samples)

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        if self._many:
            combo = self.many_(backend,
                               self.numbers,
                               self.max_numbers)

            if self.extra:
                extra = self.many_(backend,
                                   self.extra,
                                   self.max_extra)

        else:
            combo = backend(self.numbers, self.max_numbers)

            if self.extra:
                extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(extra))  # type: ignore

In [2]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [3]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 04/06/2023 11:04 
numbers combo: 15 16 23 28 31
Superstar: 11 12
backend: choice
######
Estrazione del: 04/06/2023 11:04 
numbers combo: 6 13 23 25 37
Superstar: 8 9
backend: int
######
Estrazione del: 04/06/2023 11:04 
numbers combo: 19 28 31 34 41
Superstar: 4 11
backend: sample

################# many ####################

######
Estrazione del: 04/06/2023 11:04 
numbers combo: 15 26 32 45 48
Superstar: 5 7
backend: choice
######
Estrazione del: 04/06/2023 11:04 
numbers combo: 9 26 27 29 30
Superstar: 8 11
backend: int
######
Estrazione del: 04/06/2023 11:04 
numbers combo: 7 19 23 37 41
Superstar: 3 5
backend: sample


In [4]:
superenalotto = Lotteria()

In [5]:
superenalotto()

Estrazione del: 04/06/2023 11:04 
numbers combo: 53 60 69 78 82 85
Superstar: 30


In [6]:
superenalotto(backend='choice')

Estrazione del: 04/06/2023 11:04 
numbers combo: 3 21 50 59 64 64
Superstar: 88


In [7]:
superenalotto.many

True

In [8]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 04/06/2023 11:04 
numbers combo: 3 4 5 6 7 8 12 15 16 18
Superstar: 12


In [9]:
winForLife.many

False

In [10]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter